In [1]:
import pandas as pd 
import numpy as np

In [9]:
data = pd.read_csv('/home/jean/School/columbia/spring24/capstone/broken-banner-detection/datasets/A_data_heatmap_300x250_3p.csv')
metrics = pd.read_csv('/home/jean/School/columbia/spring24/capstone/broken-banner-detection/datasets/A_data_domain_perf_metrics_300x250_3p.csv')

In [8]:
heatmap.head(3) # domain	grid_id	click_x	click_y	display_height	display_width	clicks
# print(len(heatmap.index)) # 1935426 = 1.9 M

,Unnamed: 0,domain,grid_id,click_x,click_y,display_height,display_width,clicks
0,0,ID_1763,333519,179,4,250,300,1
1,1,ID_1763,333519,262,185,250,300,1
2,2,ID_1763,333519,122,188,250,300,1


In [12]:
data_heatmap_expanded = data.loc[data.index.repeat(data['clicks'])].reset_index(drop=True)
data_heatmap_expanded
# Repeats Rows Based on 'clicks' Column: It uses the repeat method on the DataFrame's index (data.index.repeat(data['clicks'])) to repeat each row a number of times specified by the corresponding value in the 'clicks' column. This effectively expands the DataFrame by duplicating rows based on the 'clicks' values.
# Selects Repeated Rows: The .loc accessor is used to select rows from the DataFrame based on the repeated index. This results in a DataFrame where each row is repeated according to its 'clicks' value.
# Resets Index: Finally, .reset_index(drop=True) is called to reset the index of the resulting DataFrame. The drop=True argument ensures that the old index is not added as a new column in the DataFrame.

,Unnamed: 0,domain,grid_id,click_x,click_y,display_height,display_width,clicks
0,0,ID_1763,333519,179,4,250,300,1
1,1,ID_1763,333519,262,185,250,300,1
2,2,ID_1763,333519,122,188,250,300,1
3,3,ID_1763,333519,263,197,250,300,1
4,4,ID_1763,333519,242,159,250,300,1
...,...,...,...,...,...,...,...,...
2584618,1935425,ID_1501,333519,136,107,250,300,8
2584619,1935425,ID_1501,333519,136,107,250,300,8
2584620,1935425,ID_1501,333519,136,107,250,300,8
2584621,1935425,ID_1501,333519,136,107,250,300,8


In [14]:
data_heatmap_expanded['display_width'].unique()

array([300])

In [ ]:
data_heatmap_expanded['clicks'] = 1 # repeat rows and set number of clicks to 1 on each row

click_stat = data_heatmap_expanded.groupby(['click_x','click_y'])['clicks'].count().reset_index() 

# %%
# Binning clicks in nr_of_x_bins, nr_of_y_bins:

NR_OF_X_BINS = 61
NR_OF_Y_BINS = 51

max_width = data['display_width'].max()
max_height = data['display_height'].max()

width_bins = np.linspace(1, max_width, NR_OF_X_BINS)
height_bins = np.linspace(1, max_height, NR_OF_Y_BINS)

width_bins_max = len(width_bins)-2 # start at 0
height_bins_max = len(height_bins)-2

data_heatmap_expanded['click_x_bin'] = pd.cut(data_heatmap_expanded['click_x'], 
                                    bins = width_bins, 
                                    labels=False, 
                                    include_lowest=True)
# The pd.cut function is a powerful tool for segmenting and sorting data values into bins, which can be particularly useful for converting continuous variables into categorical ones for analysis or visualization purposes, such as creating a heatmap

data_heatmap_expanded['click_y_bin'] = pd.cut(data_heatmap_expanded['click_y'], 
                                    bins=height_bins, 
                                    labels=False, 
                                    include_lowest=True)


aggregated_clicks = data_heatmap_expanded.groupby(['grid_id',
                                        'domain', 
                                        'click_x_bin', 
                                        'click_y_bin']).size().reset_index(name='clicks_sum')




In [7]:
metrics.head(3) # domain	grid_id	webview_height	webview_width	displays	clicks	landed_clicks	non_bounced_clicks	closing_events	avg_last_second_framerate	sov_short_ttc	sov_short_ttc_global	sov_short_ttc_score
# print(len(metrics.index)) # 5506

,Unnamed: 0,domain,grid_id,webview_height,webview_width,displays,clicks,landed_clicks,non_bounced_clicks,closing_events,avg_last_second_framerate,sov_short_ttc,sov_short_ttc_global,sov_short_ttc_score
0,0,ID_1815,333519,250.0,300.0,2832,11.0,6.0,3.0,7.0,60.578947,0.002750,0.049334,-0.046584
1,1,ID_979,333519,250.0,300.0,2169,1.0,1.0,1.0,NaN,59.000000,NaN,NaN,NaN
2,2,ID_1413,333519,250.0,300.0,7077,16.0,2.0,0.0,5.0,31.333333,0.345627,0.185044,0.292426
